In [1]:
import polaris as po
import numpy as np

# Load the competition from the Hub
competition = po.load_competition("asap-discovery/antiviral-ligand-poses-2025")

#recommended
competition.cache()

# Get the train and test data-loaders
train, test = competition.get_train_test_split()

[2025-03-12 15:27:39] INFO     The version of Polaris that was used to create the artifact (0.11.0) ]8;id=734225;file:///home/wim/anaconda3/envs/my-rdkit-env/lib/python3.12/site-packages/polaris/_artifact.py\_artifact.py]8;;\:]8;id=556889;file:///home/wim/anaconda3/envs/my-rdkit-env/lib/python3.12/site-packages/polaris/_artifact.py#92\92]8;;\
                               is different from the currently installed version of Polaris                        
                               (0.11.9).                                                                           

                      WARNING  You're loading data from a remote location. If the dataset is small     ]8;id=360017;file:///home/wim/anaconda3/envs/my-rdkit-env/lib/python3.12/site-packages/polaris/dataset/_base.py\_base.py]8;;\:]8;id=213814;file:///home/wim/anaconda3/envs/my-rdkit-env/lib/python3.12/site-packages/polaris/dataset/_base.py#181\181]8;;\
                               enough, consider caching the dataset first using                                    
                               CompetitionSpecification.cache() for more performant data access.                   

Output()

[15:27:40] Copying to destination                                                                      ]8;id=817561;file:///home/wim/anaconda3/envs/my-rdkit-env/lib/python3.12/site-packages/polaris/dataset/_base.py\_base.py]8;;\:]8;id=348142;file:///home/wim/anaconda3/envs/my-rdkit-env/lib/python3.12/site-packages/polaris/dataset/_base.py#380\380]8;;\
           /home/wim/.cache/polaris/datasets/a66af4e0-ce18-4ebb-8fd6-a0c89c230c91/data.zarr                        

           For large Zarr archives, this may take a while.                                             ]8;id=772805;file:///home/wim/anaconda3/envs/my-rdkit-env/lib/python3.12/site-packages/polaris/dataset/_base.py\_base.py]8;;\:]8;id=396624;file:///home/wim/anaconda3/envs/my-rdkit-env/lib/python3.12/site-packages/polaris/dataset/_base.py#381\381]8;;\

[15:28:05]  Success: Copying Zarr keys                                                                ]8;id=950856;file:///home/wim/anaconda3/envs/my-rdkit-env/lib/python3.12/site-packages/polaris/utils/context.py\context.py]8;;\:]8;id=291644;file:///home/wim/anaconda3/envs/my-rdkit-env/lib/python3.12/site-packages/polaris/utils/context.py#53\53]8;;\

            Success: Copying Zarr archive                                                             ]8;id=410849;file:///home/wim/anaconda3/envs/my-rdkit-env/lib/python3.12/site-packages/polaris/utils/context.py\context.py]8;;\:]8;id=766763;file:///home/wim/anaconda3/envs/my-rdkit-env/lib/python3.12/site-packages/polaris/utils/context.py#53\53]8;;\

In [2]:
import biotite.structure as struc
import biotite.structure.io.pdb as pdb
from biotite.structure import superimpose_homologs
from rdkit import Chem
from rdkit.Chem import AllChem
import tempfile

sarsmols = []


for i in range(len(train)):
    # Load the reference PDB structure
    ref_pdb = pdb.PDBFile.read("ref_structs/sarsA.pdb")
    ref_atoms = ref_pdb.get_structure()
    complex_atoms = competition[i]["Complex Structure"]
    amino_acid_residues = [
        "ALA", "ARG", "ASN", "ASP", "CYS", "GLU", "GLN", "GLY", "HIS", "ILE",
        "LEU", "LYS", "MET", "PHE", "PRO", "SER", "THR", "TRP", "TYR", "VAL"
    ]
    print(complex_atoms.res_name)
    
    # Identify ligand atoms (non-standard residues)
    rightchain = competition[i]['Ligand Pose'].GetProp("_Name")[4]
    is_ligand = ~np.isin(complex_atoms.res_name, amino_acid_residues)
    
    # Identify atoms belonging to Chain A
    is_chain_a = complex_atoms.chain_id == rightchain
    
    # Keep Chain A **OR** ligands
    complex_atoms = complex_atoms[is_chain_a | is_ligand]
    
    
    fitted,_,_,_ = superimpose_homologs(ref_atoms, complex_atoms)
    is_ligand = ~np.isin(fitted.res_name, amino_acid_residues)
    ligand_atoms = fitted[is_ligand]
    
    # Create a temporary PDB file
    with tempfile.NamedTemporaryFile(suffix=".pdb", delete=False) as temp_pdb:
        out_pdb = pdb.PDBFile()
        out_pdb.set_structure(ligand_atoms)
        out_pdb.write(temp_pdb.name)
        temp_pdb_path = temp_pdb.name  # Store filename
    
    # Convert PDB to RDKit Mol object
    tmp = Chem.MolFromPDBFile(temp_pdb_path)
    template = Chem.MolFromSmiles(competition[i]["CXSMILES"])
    mol = AllChem.AssignBondOrdersFromTemplate(template, tmp)
    sarsmols.append(mol)
    print(i)
wri = Chem.SDWriter("sarsmols.sdf")
for m in sarsmols:
    wri.write(m)
wri.close()

['SER' 'SER' 'SER' ... 'UNK' 'UNK' 'UNK']


[15:28:09] WARNING: More than one matching pattern found - picking one



0
['SER' 'SER' 'SER' ... 'UNK' 'UNK' 'UNK']
1
['SER' 'SER' 'SER' ... 'UNK' 'UNK' 'UNK']


[15:28:12] WARNING: More than one matching pattern found - picking one



2
['SER' 'SER' 'SER' ... 'UNK' 'UNK' 'UNK']
3
['SER' 'SER' 'SER' ... 'UNK' 'UNK' 'UNK']


...

[15:51:04] WARNING: More than one matching pattern found - picking one



766
['SER' 'SER' 'SER' ... 'UNK' 'UNK' 'UNK']
767
['SER' 'SER' 'SER' ... 'UNK' 'UNK' 'UNK']


[15:51:07] WARNING: More than one matching pattern found - picking one



768
['SER' 'SER' 'SER' ... 'UNK' 'UNK' 'UNK']
769


[15:51:08] WARNING: More than one matching pattern found - picking one

